# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from IPython import display
import warnings

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName;", engine)
X = df.message.values
Y = df.iloc[:, 4:].values
category_label = df.iloc[:, 4:].columns.tolist()

In [3]:
df[df.genre=='direct'].sample(15)[['genre', 'message']].values

array([['direct', 'Why does it rain every night? '],
       ['direct', 'What do I need to do to find a tent? '],
       ['direct', 'we are hungra please help us now otherwise we die '],
       ['direct',
        "The internet caf Net@le that's by the Dal road by the Maranata church ( incomplete )"],
       ['direct',
        '. ..  Esdras. Address : Solidarit Village, Toussain Louverture Street 83A!'],
       ['direct',
        'How many dollars the French willing to give Haiti about debts of Independence? '],
       ['direct',
        'Good Day, I am happy your desk accepted my message. I am a student, I would like to get a job, I am in misery'],
       ['direct', 'thanks for all your advice that you give me. '],
       ['direct',
        'If a pregnant woman is vomiting blood, what should we do?'],
       ['direct',
        'please please i pray that God sees what misery we are in please help us '],
       ['direct',
        'What conditions are necessary for a Haitian to go to Seneg

In [23]:
def silence_print(ture_label, pred_label, silence=True):
    """Print classification report without warning message
    
    Parameters:
    -----------
    true_label: array
        The true values
    pred_label: array
        The predict values
    silence: boolean default True
        If true, print the report without warning message
    """
    if silence:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            for index, column in enumerate(category_label):
                print(classification_report(test_y[:, index], pred_y[:, index], target_names=[column]))
    else:
        for index, column in enumerate(category_label):
            print(classification_report(test_y[:, index], pred_y[:, index], target_names=[column]))

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # nromalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=2))
])

### 4. Train pipeline
* Split data into train and test sets
* Train pipeline

In [6]:
# split data
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=42)

In [7]:
# train data
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=2))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# basic model
pred_y = pipeline.predict(test_X)

In [9]:
pred_y[:4, 0]

array([1, 1, 1, 1], dtype=int64)

In [13]:
# print the report
silence_print(test_y, pred_y)

             precision    recall  f1-score   support

    related       0.60      0.45      0.52      1873

avg / total       0.78      0.79      0.78      7865

             precision    recall  f1-score   support

    request       0.90      0.97      0.93      6533

avg / total       0.88      0.89      0.87      7865

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      7829

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

aid_related       0.76      0.84      0.80      4646

avg / total       0.75      0.75      0.75      7865

              precision    recall  f1-score   support

medical_help       0.93      0.99      0.96      7227

 avg / total       0.90      0.92      0.90      7865

                  precision    recall  f1-score   support

medical_products       0.95      1.00      0.97      7447

     avg / total       0.95      0.95      0.93      7865

          

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
forest = RandomForestClassifier(random_state=42)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest))
])

In [20]:
# params
params = {
#     'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'text_pipeline__vect__max_df': (0.75, 1.0),
    'text_pipeline__vect__max_features': (5000, 100000),
#     'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [4, 6, 10],

}

In [21]:
# train model
cv = GridSearchCV(pipeline, param_grid=params)

cv.fit(train_X, train_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'text_pipeline__vect__max_features': (5000, 100000), 'clf__estimator__n_estimators': [10, 20, 30], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [4, 6, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
# predict values
pred_y = cv.predict(test_X)

In [24]:
# print report
silence_print(test_y, pred_y)

             precision    recall  f1-score   support

    related       0.00      0.00      0.00      1873

avg / total       0.57      0.75      0.65      7865

             precision    recall  f1-score   support

    request       0.83      1.00      0.91      6533

avg / total       0.69      0.83      0.75      7865

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      7829

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

aid_related       0.59      1.00      0.75      4646

avg / total       0.75      0.60      0.45      7865

              precision    recall  f1-score   support

medical_help       0.92      1.00      0.96      7227

 avg / total       0.84      0.92      0.88      7865

                  precision    recall  f1-score   support

medical_products       0.95      1.00      0.97      7447

     avg / total       0.90      0.95      0.92      7865

          

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
def chunck_tokenize(text):
    # nromalize text
    text = re.sub(r"#[a-zA-Z0-9]", "tagname", text)
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "urlholder", text)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token.lower() not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

In [26]:
def check(x):
    length = len(sent_tokenize(x))
    chunck = False

    for sentence in sent_tokenize(x):
        pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

        try:
            word, tag = pos_tags[0]

            if "VB" in tag:
                chunck = True
        except:
            continue
        
        try:
            for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                if tag == "NNP":
                    chunck = True
                    break
        except:
            continue
            
        if chunck:
            break

    return (length, chunck)

In [27]:
class WordChunckExtract(BaseEstimator, TransformerMixin):
    def starting_verb(self, text): 
        length = len(sent_tokenize(text))
        chunck = False

        for sentence in sent_tokenize(text):
            pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

            try:
                word, tag = pos_tags[0]

                if "VB" in tag:
                    chunck = True
            except:
                continue

            try:
                for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                    if tag == "NNP":
                        chunck = True
                        break
            except:
                continue

            if chunck:
                break

        return (length, chunck)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        result = pd.DataFrame()
        X_tagged = pd.Series(X).apply(self.starting_verb)
        result["length"] = X_tagged.apply(lambda x: x[0] > 1)
        result["chucnk"] = X_tagged.apply(lambda x: x[1])
        return result


In [28]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text_pipeline", Pipeline([
            ("vect", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())
        ])), 
        ("exact_chunck", WordChunckExtract())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=4))
])

In [29]:
# params
params = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [100, 150],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [12, 18, 25],
    'features__transformer_weights': (
    {'text_pipeline': 1, 'starting_verb': 0.5},
#     {'text_pipeline': 0.5, 'starting_verb': 1},
#     {'text_pipeline': 0.8, 'starting_verb': 1},
)

}

# train model
cv = GridSearchCV(pipeline, param_grid=params)

cv.fit(train_X, train_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=4))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'clf__estimator__n_estimators': [100, 150], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [12, 18, 25], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5},)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
# predict values
pred_y = cv.predict(test_X)

In [31]:
silence_print(test_y, pred_y)

             precision    recall  f1-score   support

    related       0.00      0.00      0.00      1873

avg / total       0.57      0.75      0.65      7865

             precision    recall  f1-score   support

    request       0.83      1.00      0.91      6533

avg / total       0.69      0.83      0.75      7865

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      7829

avg / total       0.99      1.00      0.99      7865

             precision    recall  f1-score   support

aid_related       0.62      0.99      0.76      4646

avg / total       0.75      0.63      0.53      7865

              precision    recall  f1-score   support

medical_help       0.92      1.00      0.96      7227

 avg / total       0.84      0.92      0.88      7865

                  precision    recall  f1-score   support

medical_products       0.95      1.00      0.97      7447

     avg / total       0.90      0.95      0.92      7865

          

### 9. Export your model as a pickle file

In [32]:
import pickle

In [33]:
with open("model.pickle", "wb") as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.